<a href="https://colab.research.google.com/github/kairamilanifitria/PurpleBox-Intern/blob/main/02_06_Colpali%20%2B%20Try%20integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# try2

In [1]:
!pip install accelerate torch>=2.0.0 torchvision einops tensorflow-cpu \
    git+https://github.com/huggingface/transformers \
    huggingface_hub[hf_transfer] gradio Pillow pydantic qwen-vl-utils[decord]==0.0.8

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ryyj7t20


In [2]:
import subprocess  # 🥲

subprocess.run(
    "pip install flash-attn --no-build-isolation",
    env={"FLASH_ATTENTION_SKIP_CUDA_BUILD": "TRUE"},
    shell=True,
)
import gradio as gr
import re

from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info
import torch
import os
import json
from pydantic import BaseModel
from typing import Tuple

In [3]:
# Install flash-attn without CUDA build
env_vars = os.environ.copy()
env_vars["FLASH_ATTENTION_SKIP_CUDA_BUILD"] = "TRUE"
subprocess.run("pip install flash-attn --no-build-isolation", shell=True, env=env_vars)

os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# Load model and processor
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-7B-Instruct",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2",
    device_map="auto",
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/57.6k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [7]:
class GeneralRetrievalQuery(BaseModel):
    broad_topical_query: str
    broad_topical_explanation: str
    specific_detail_query: str
    specific_detail_explanation: str
    visual_element_query: str
    visual_element_explanation: str

def extract_json_with_regex(text):
    pattern = r'```(?:json)?\s*(.+?)\s*```'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches[0] if matches else None

def get_retrieval_prompt(prompt_name: str) -> Tuple[str, GeneralRetrievalQuery]:
    if prompt_name != "general":
        raise ValueError("Only 'general' prompt is available in this version")

    prompt = """You are an AI assistant specialized in document retrieval tasks. Given an image of a document page, your task is to generate retrieval queries that someone might use to find this document in a large corpus.

Please generate 3 different types of retrieval queries:

1. A broad topical query: This should cover the main subject of the document.
2. A specific detail query: This should focus on a particular fact, figure, or point made in the document.
3. A visual element query: This should reference a chart, graph, image, or other visual component in the document, if present. Dont forget to make a reference the name of the visual element also generate a query which this illustration may help answer or be related to.

Important guidelines:
- Ensure the queries are relevant for retrieval tasks, but dont forget for describing the page content.
- Frame the queries as if someone is searching for this document, not asking questions about its content.
- Make the queries diverse and representative of different search strategies.

For each query, also provide a brief explanation of why this query would be effective in retrieving this document.

Format your response as a JSON object with the following structure:

{
  "broad_topical_query": "Your query here",
  "broad_topical_explanation": "Brief explanation",
  "specific_detail_query": "Your query here",
  "specific_detail_explanation": "Brief explanation",
  "visual_element_query": "Your query here",
  "visual_element_explanation": "Brief explanation"
}

If there are no relevant visual elements, replace the third query with another specific detail query.

Here is the document image to analyze:
<image>

Generate the queries based on this image and provide the response in the specified JSON format."""
    return prompt, GeneralRetrievalQuery

prompt, pydantic_model = get_retrieval_prompt("general")

def _prep_data_for_input(image):
    messages = [{"role": "user", "content": [{"type": "image", "image": image}, {"type": "text", "text": prompt}]}]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, video_inputs = process_vision_info(messages)
    return processor(text=[text], images=image_inputs, videos=video_inputs, padding=True, return_tensors="pt")

def generate_response(image):
    inputs = _prep_data_for_input(image).to("cuda")
    generated_ids = model.generate(**inputs, max_new_tokens=200)
    generated_ids_trimmed = [out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    try:
        json_str = extract_json_with_regex(output_text)
        return json.dumps(json.loads(json_str) if json_str else json.loads(output_text), indent=2)
    except Exception:
        return output_text

demo = gr.Interface(
    fn=generate_response,
    inputs=gr.Image(type="pil"),
    outputs=gr.Text(),
    title="ColPali Query Generator using Qwen2.5-VL",
)
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d42055ea1a3409af2f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [12]:
import base64
import requests
import json
import gradio as gr
import io
from PIL import Image

GROQ_API_KEY = "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"
GROQ_MODEL = "llama-3.2-90b-vision-preview"  # Change model as needed

def encode_image(image):
    """Convert image to base64 for API request."""
    buffered = io.BytesIO()
    image.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")

def generate_retrieval_query():
    """Create the prompt for document retrieval queries."""
    return """do the document retrieval tasks. Given an image of a document page, generate 3 different types of retrieval queries:

1. Broad topical query: main subject of the document.
2. Specific detail query: fact, figure, or point in the document.
3. Visual element query: References a chart, graph, image, or other visual component.
""", None

def generate_response(image):
    """Send request to Groq API to generate document queries."""
    image_b64 = encode_image(image)
    prompt, _ = generate_retrieval_query()

    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": GROQ_MODEL,
        "messages": [
            {"role": "system", "content": "You are an AI specialized in document retrieval."},
            {"role": "user", "content": "Analyze this document image."},
            {"role": "user", "content": f"Here is the image (base64): {image_b64}"},
            {"role": "user", "content": prompt},
        ],
        "max_tokens": 500
    }

    # ✅ Correct API URL
    api_url = "https://api.groq.com/openai/v1/chat/completions"

    response = requests.post(api_url, headers=headers, json=payload)

    try:
        response_json = response.json()
        print("DEBUG API Response:", json.dumps(response_json, indent=2))  # Print API response

        if "choices" in response_json and response_json["choices"]:
            return response_json["choices"][0]["message"]["content"]
        else:
            return f"Error: Unexpected API response structure: {response_json}"

    except Exception as e:
        return f"Error: Unable to parse response. Exception: {str(e)}"

title = "LLaMA Query Generator via Groq API"
description = "This interface uses LLaMA-3 on the Groq API to generate retrieval queries for documents based on an input image."

demo = gr.Interface(
    fn=generate_response,
    inputs=gr.Image(type="pil"),
    outputs=gr.Text(),
    title=title,
    description=description,
)
demo.launch()



Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://286ee43c94b5e07fe4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Docling Conversion

In [ ]:
import logging
import time
import warnings

warnings.filterwarnings("ignore")

# Set up logging
logging.basicConfig(level=logging.INFO)
_log = logging.getLogger(__name__)

In [ ]:
!pip install --upgrade --force-reinstall torch torchvision torchaudio

In [ ]:
import torch

# Check if GPU or MPS is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"CUDA GPU is enabled: {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS GPU is enabled.")
else:
    raise EnvironmentError(
        "No GPU or MPS device found. Please check your environment and ensure GPU or MPS support is configured."
    )

CUDA GPU is enabled: Tesla T4


In [ ]:
# !pip install transformers accelerate numpy Requests torch torchvision qwen-vl-utils av ipython reportlab fpdf python-docx pillow huggingface_hub
!pip install llama-index>=0.12.8 llama-index-core>=0.12.8
!pip install llama-index-node-parser-docling>=0.3.0 llama-index-readers-docling>=0.3.0
!pip install pypdf2>=3.0.1
!pip install easyocr>=1.7.2

# !pip install ipykernel>=6.29.5
# !pip install llama-index-embeddings-ollama>=0.5.0 llama-index-embeddings-huggingface>=0.4.0
# !pip install llama-index-llms-huggingface-api>=0.3.0 llama-index-llms-ollama>=0.5.0
# !pip install llama-index-readers-file>=0.4.1
# !pip install llama-index-vector-stores-milvus>=0.4.0
# !pip install python-dotenv>=1.0.1
# !pip install rich>=13.9.4
# !pip install pillow>=10.4.0
# !pip install pyarrow>=18.1.0
# !pip install fastparquet>=2024.11.0
# !pip install datasets>=3.2.0
# #!pip install ocrmac>=1.0.0
# !pip install matplotlib>=3.10.0
# !pip install toml>=0.10.2

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.18 requires torch<2.6,>=1.10, but you have torch 2.6.0 which is incompatible.


In [ ]:
# English Files
# Upload all the files to Google Colab Files. The code below will create a directory called 'data' and move the files to it.
!mkdir -p data

!mv 17.pdf data/17.pdf
!mv 2014-monarch-plus-service-manual.pdf data/2014-monarch-plus-service-manual.pdf
!mv 231161_OperationsMaintenanceManual.docx data/231161_OperationsMaintenanceManual.docx
!mv PDF1.pdf data/PDF1.pdf
!mv SUPO-744_REV_A.pdf data/SUPO-744_REV_A.pdf
!mv VVS005s_030s_AHU_EN.pdf data/VVS005s_030s_AHU_EN.pdf
!mv ai-in-america-oai-economic-blueprint-20250113.pdf data/ai-in-america-oai-economic-blueprint-20250113.pdf
!mv creatingsystem.pdf data/creatingsystem.pdf

In [ ]:
import time
from docling.datamodel.base_models import InputFormat
from docling_core.types.doc import ImageRefMode
from docling.document_converter import DocumentConverter, PdfFormatOption, WordFormatOption
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend  # Import the backend
from docling.pipeline.standard_pdf_pipeline import StandardPdfPipeline # Import the Pipeline
from docling.datamodel.pipeline_options import PdfPipelineOptions, TableFormerMode, EasyOcrOptions, TesseractOcrOptions, OcrMacOptions
from docling.pipeline.simple_pipeline import SimplePipeline
from docling.datamodel.settings import settings

IMAGE_RESOLUTION_SCALE = 2.0

def create_pipeline_options(input_format):
    """Creates dynamic pipeline options based on the input format."""
    if input_format == InputFormat.PDF:

        return PdfFormatOption(
            pipeline_options = PdfPipelineOptions(
                do_table_structure=True,  # Enable table structure detection
                # do_ocr=True,  # Enable OCR
                # ocr_options=EasyOcrOptions(force_full_page_ocr=True, lang=["en"]),
                # table_structure_option=dict(
                #     do_cell_matching=True,
                #     mode=TableFormerMode.ACCURATE
                # ),
                generate_page_images=True,
                generate_picture_images=True,
                images_scale=IMAGE_RESOLUTION_SCALE,
            )

            # pipeline_cls=StandardPdfPipeline,  # Specify the pipeline class
            # backend=PyPdfiumDocumentBackend,  # Specify the backend explicitly
            # do_table_structure=True,  # Enable table structure detection

            # generate_page_images=True,
            # generate_picture_images=True,
            # image_mode=ImageRefMode.EMBEDDED,  # or ImageRefMode.REFERENCED
            # images_scale=IMAGE_RESOLUTION_SCALE,
        )

    elif input_format == InputFormat.DOCX:
        return WordFormatOption(
            pipeline_cls=SimplePipeline  # Configure Word document pipeline
        )

    elif input_format == InputFormat.IMAGE:
        return None  # Add image-specific options if needed
    elif input_format == InputFormat.HTML:
        return None  # Add HTML-specific options if needed
    elif input_format == InputFormat.PPTX:
        return None  # Add PowerPoint-specific options if needed
    elif input_format == InputFormat.ASCIIDOC or input_format == InputFormat.MD:
        return None  # These formats might not need advanced options
    else:
        raise ValueError(f"Unsupported input format: {input_format}")

In [ ]:
from docling.document_converter import DocumentConverter

def initialize_converter():
    """Initializes the document converter with multiformat support and advanced options."""
    allowed_formats = [
        InputFormat.PDF,
        InputFormat.IMAGE,
        InputFormat.DOCX,
        InputFormat.HTML,
        InputFormat.PPTX,
        InputFormat.ASCIIDOC,
        InputFormat.MD,
    ]

    format_options = {
        input_format: create_pipeline_options(input_format)
        for input_format in allowed_formats
        if create_pipeline_options(input_format) is not None
    }

    return DocumentConverter(
        allowed_formats=allowed_formats,
        format_options=format_options,
    )


In [ ]:
from pathlib import Path
import json
import yaml
import logging  # Import logging here

def convert_and_save(input_paths, output_dir, image_mode=ImageRefMode.REFERENCED): # ImageRefMode.EMBEDDED or ImageRefMode.REFERENCED
    """Converts documents to Markdown and saves the output."""
    _log = logging.getLogger(__name__)  # Define _log inside the function
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    doc_converter = initialize_converter()
    conv_results = doc_converter.convert_all(input_paths)

    for res in conv_results:
        file_name = res.input.file.stem
        markdown_path = output_dir / f"{file_name}.md"
        json_path = output_dir / f"{file_name}.json"
        yaml_path = output_dir / f"{file_name}.yaml"

        # Save Markdown output
        res.document.save_as_markdown(markdown_path, image_mode=image_mode)
        _log.info(f"Markdown content saved to {markdown_path}")


In [ ]:
import json
from pathlib import Path

def extract_all_nodes_with_image_refs(md_file_path, output_dir):
    """
    Extracts all nodes from a markdown file, including image references.
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    output_path = output_dir / f"{md_file_path.stem}_all_nodes_with_images.json"

    try:
        # Try reading with 'utf-8' first
        with open(md_file_path, 'r', encoding='utf-8') as f:
            markdown_content = f.read()
    except UnicodeDecodeError:
        # If 'utf-8' fails, try 'latin-1'
        try:
            with open(md_file_path, 'r', encoding='latin-1') as f:
                markdown_content = f.read()
            print(f"Warning: File {md_file_path} decoded using 'latin-1' due to UTF-8 errors.")
        except UnicodeDecodeError:
            print(f"Error: Could not decode file {md_file_path} using either 'utf-8' or 'latin-1'.")
            return
    except FileNotFoundError:
        print(f"Error: File not found at {md_file_path}")
        return

    all_nodes = []
    current_text_block = ""

    lines = markdown_content.split('\n')
    for i, line in enumerate(lines):
        if '![' in line and ']' in line and '(' in line and ')' in line:
            # Found an image reference
            image_ref_line = line
            parts = image_ref_line.split('(')
            if len(parts) >= 2:
                image_path = parts[1].split(')')[0]
                node_text = parts[0].split('[')[1].split(']')[0]

                # Add previous text block as a node if it's not empty
                if current_text_block.strip():
                    all_nodes.append({
                        "index": len(all_nodes) + 1,
                        "text": current_text_block.strip(),
                        "image_path": None  # Indicate no image for this node
                    })

                # Add the image node
                all_nodes.append({
                    "index": len(all_nodes) + 1,
                    "text": node_text,
                    "image_path": image_path
                })
                current_text_block = ""  # Reset text block
        else:
            # Accumulate text for non-image lines
            current_text_block += line + "\n"

    # Add the final text block if it's not empty
    if current_text_block.strip():
        all_nodes.append({
            "index": len(all_nodes) + 1,
            "text": current_text_block.strip(),
            "image_path": None
        })

    extracted_data = {
        "file_name": md_file_path.name,
        "number_of_nodes": len(all_nodes),
        "nodes": all_nodes
    }

    with output_path.open("w") as fp:
        json.dump(extracted_data, fp, indent=4)

    print(f"Extracted {len(all_nodes)} nodes from {md_file_path.name} and saved to {output_path}")

In [ ]:
def main():
    settings.debug.profile_pipeline_timings = True

    input_paths = [
        Path("data/17.pdf"),
        Path("data/2014-monarch-plus-service-manual.pdf"),
        Path("data/231161_OperationsMaintenanceManual.docx"),
        Path("data/PDF1.pdf"),
        Path("data/SUPO-744_REV_A.pdf"),
        Path("data/ai-in-america-oai-economic-blueprint-20250113.pdf"),
        Path("data/creatingsystem.pdf"),
    ]

    output_dir = "output-docs"
    convert_and_save(input_paths, output_dir)

    # Process only Markdown files
    md_files = list(Path(output_dir).glob("*.md"))

    for md_file in md_files:
        extract_all_nodes_with_image_refs(md_file, output_dir)



In [ ]:
if __name__ == "__main__":
    main()

Could not load the custom kernel for multi-scale deformable attention: /root/.cache/torch_extensions/py311_cu124/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /root/.cache/torch_extensions/py311_cu124/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /root/.cache/torch_extensions/py311_cu124/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable attention: /root/.cache/torch_extensions/py311_cu124/MultiScaleDeformableAttention/MultiScaleDeformableAttention.so: cannot open shared object file: No such file or directory
Could not load the custom kernel for multi-scale deformable 

Extracted 43 nodes from ai-in-america-oai-economic-blueprint-20250113.md and saved to output-docs/ai-in-america-oai-economic-blueprint-20250113_all_nodes_with_images.json
Extracted 2 nodes from creatingsystem.md and saved to output-docs/creatingsystem_all_nodes_with_images.json
Extracted 72 nodes from SUPO-744_REV_A.md and saved to output-docs/SUPO-744_REV_A_all_nodes_with_images.json
Extracted 17 nodes from PDF1.md and saved to output-docs/PDF1_all_nodes_with_images.json
Extracted 3 nodes from 231161_OperationsMaintenanceManual.md and saved to output-docs/231161_OperationsMaintenanceManual_all_nodes_with_images.json
Extracted 11 nodes from 17.md and saved to output-docs/17_all_nodes_with_images.json
Extracted 280 nodes from 2014-monarch-plus-service-manual.md and saved to output-docs/2014-monarch-plus-service-manual_all_nodes_with_images.json


# Image extraction from nodes + Qwen processing functions

In [ ]:
import os
os.environ["PYTHONUTF8"] = "1"
os.environ["LC_ALL"] = "C.UTF-8"
os.environ["LANG"] = "C.UTF-8"
os.environ["LANGUAGE"] = "C.UTF-8"

In [ ]:
!pip install transformers accelerate numpy Requests torch torchvision qwen-vl-utils av ipython reportlab fpdf python-docx pillow huggingface_hub --quiet
!pip install git+https://github.com/huggingface/transformers accelerate --quiet

In [ ]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, TextIteratorStreamer
from qwen_vl_utils import process_vision_info
from PIL import Image
import uuid
import io
from threading import Thread
# from reportlab.lib.pagesizes import A4
# from reportlab.lib.styles import getSampleStyleSheet
# from reportlab.platypus import SimpleDocTemplate, Image as RLImage, Paragraph, Spacer
# from reportlab.lib.units import inch
# from reportlab.pdfbase import pdfmetrics
# from reportlab.pdfbase.ttfonts import TTFont
import docx
# from docx.enum.text import WD_ALIGN_PARAGRAPH
import json

In [ ]:
# Load the model and processor
model_name = "Qwen/Qwen2-VL-2B-Instruct"

model = Qwen2VLForConditionalGeneration.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.float32).to("cpu").eval()

processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

image_extensions = Image.registered_extensions()


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
import base64
def identify_and_save_blob(blob_path):
    """Identifies if the blob is an image and saves it."""
    try:
        with open(blob_path, 'rb') as file:
            blob_content = file.read()
            try:
                Image.open(io.BytesIO(blob_content)).verify()  # Check if it's a valid image
                extension = ".png"  # Default to PNG for saving
                media_type = "image"
            except (IOError, SyntaxError):
                raise ValueError("Unsupported media type. Please upload a valid image.")

            filename = f"temp_{uuid.uuid4()}_media{extension}"
            with open(filename, "wb") as f:
                f.write(blob_content)

            return filename, media_type

    except FileNotFoundError:
        raise ValueError(f"The file {blob_path} was not found.")
    except Exception as e:
        raise ValueError(f"An error occurred while processing the file: {e}")

def decode_base64_to_image(base64_string):
    """Decodes a base64 string and saves it as a temporary image file."""
    image_data = base64.b64decode(base64_string)
    image = Image.open(io.BytesIO(image_data))

    filename = f"temp_{uuid.uuid4()}.png"  # Always save as PNG
    image.save(filename)

    return filename

def qwen_inference(media_input, text_input=None):
    torch.cuda.empty_cache()
    # torch.cuda.memory_summary(device=None, abbreviated=False)
    """Handles inference for the selected model."""
    if media_input.startswith("data:image"):  # Detect base64 input
            base64_str = media_input.split(",")[1]  # Remove the base64 header
            media_path = decode_base64_to_image(base64_str)
    elif media_input.endswith(tuple(image_extensions.keys())):  # If it's a file path
            media_path = media_input
    else:
            raise ValueError("Unsupported media type. Please provide a valid image path or base64 string.")
    media_type = "image"
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": media_type,
                    media_type: media_path
                },
                {"type": "text", "text": text_input},
            ],
        }
    ]

    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, _ = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        padding=True,
        return_tensors="pt",
    ).to("cpu")

    # Run the model in a separate thread
    streamer = TextIteratorStreamer(
        processor.tokenizer, skip_prompt=True, skip_special_tokens=True
    )
    generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=1024)

    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    buffer = ""
    for new_text in streamer:
        buffer += new_text
        # Remove <|im_end|> or similar tokens from the output
        buffer = buffer.replace("<|im_end|>", "")
        yield buffer

def format_plain_text(output_text):
    """Formats the output text as plain text without LaTeX delimiters."""
    # Remove LaTeX delimiters and convert to plain text
    plain_text = output_text.replace("\\(", "").replace("\\)", "").replace("\\[", "").replace("\\]", "")
    return plain_text


In [ ]:
# deleting duplicate images
import hashlib
import os
import re
import glob
from PIL import Image


def calculate_hash(image_path):
    """Calculate the hash of an image."""
    try:
        with Image.open(image_path) as img:
            img = img.resize((256, 256)).convert("RGB")  # Resize and standardize
            return hashlib.md5(img.tobytes()).hexdigest()
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

def find_and_remove_duplicates(folder_path):
    """Find and remove duplicate images in the given folder."""
    if not os.path.exists(folder_path):
        print(f"The folder '{folder_path}' does not exist.")
        return

    hashes = {}
    duplicates = []

    for root, _, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif", ".tiff")):
                img_hash = calculate_hash(file_path)
                if img_hash:
                    if img_hash in hashes:
                        duplicates.append(file_path)
                    else:
                        hashes[img_hash] = file_path

    # Remove duplicates
    for duplicate in duplicates:
        try:
            os.remove(duplicate)
            print(f"Deleted: {duplicate}")
        except Exception as e:
            print(f"Could not delete {duplicate}: {e}")

    print(f"Total duplicates removed: {len(duplicates)}")

def remove_small_images(image_folder, min_size=30):
    """Deletes images smaller than min_size x min_size pixels."""
    len = 0
    for img_file in glob.glob(os.path.join(image_folder, "*.*")):
        try:
            with Image.open(img_file) as img:
                if img.size[0] < min_size or img.size[1] < min_size:
                    os.remove(img_file)
                    print(f"Deleted: {img_file}")
                    len = len+1
        except Exception:
            continue
    print(f"Total small images removed: {len}")


from PIL import Image
import os

def compress_image(image_path, max_size_kb=50, quality=85, min_quality=10):
    """Compress and resize an image to ensure its size is at most max_size_kb."""
    max_size_bytes = max_size_kb * 1024  # Convert KB to Bytes

    # Open the image
    with Image.open(image_path) as img:
        img_format = img.format  # Preserve original format
        temp_quality = quality  # Start with high quality
        width, height = img.size  # Get original dimensions

        # Step 1: Try reducing quality first
        while True:
            temp_path = image_path.replace(".", "_compressed.")
            img.save(temp_path, format=img_format, quality=temp_quality, optimize=True)

            if os.path.getsize(temp_path) <= max_size_bytes or temp_quality <= min_quality:
                break  # Stop if within size limit or quality is too low

            temp_quality -= 5  # Reduce quality step by step

        # Step 2: If still too big, start resizing
        while os.path.getsize(temp_path) > max_size_bytes:
            width = int(width * 0.9)  # Reduce width by 10%
            height = int(height * 0.9)  # Reduce height by 10%
            img = img.resize((width, height), resample=Image.LANCZOS)
            img.save(temp_path, format=img_format, quality=temp_quality, optimize=True)

        # Overwrite the original file with the compressed version
        os.replace(temp_path, image_path)
        print(f"Compressed: {image_path} to {os.path.getsize(image_path) / 1024:.2f} KB")

# Example usage:
# compress_image("example.jpg")


In [ ]:
import os
import json
import re
from PIL import Image

def update_json(json_file, json_folder):
    """Updates JSON by removing nodes with missing images and renewing indexes."""

    # Load JSON file
    with open(json_file, "r", encoding="utf-8") as file:
        data = json.load(file)

    filename = os.path.basename(json_file).replace("_all_nodes_with_images.json", "")
    # Get list of actual image files in the same folder
    artifacts_folder = filename + "_artifacts"
    img_folder = os.path.join(json_folder, artifacts_folder)
    valid_images = set(os.listdir(img_folder))  # Images that exist in json_folder


    new_nodes = []
    index_counter = 1

    for node in data["nodes"]:
        if node.get("image_path"):  # If node has an image
            image_name = os.path.basename(node["image_path"])
            if image_name not in valid_images:
                continue  # Skip nodes where the image is missing

        # Trim text field to max 3000 characters
        if "text" in node and isinstance(node["text"], str):
            node["text"] = node["text"][:3000]  # Keep only the first 3000 characters

        node["index"] = index_counter  # Update index sequentially
        new_nodes.append(node)
        index_counter += 1

    # Update JSON metadata
    data["nodes"] = new_nodes
    data["number_of_nodes"] = len(new_nodes)

    # Save new JSON file
    new_json_file = os.path.join(json_folder, os.path.basename(json_file).replace(".json", "_updated.json"))
    with open(new_json_file, "w", encoding="utf-8") as file:
        json.dump(data, file, indent=4, ensure_ascii=False)

    print(f"Updated JSON saved as: {new_json_file}")
    return new_json_file



def process_image(image_path, context_text, json_folder):
    compress_image(image_path)
    """Generate explanation for an image using context from surrounding nodes."""
    media_input = os.path.join(json_folder, image_path)  # Construct full image path

    # Check if the image file exists
    if not os.path.exists(media_input):
        return ""

    if not context_text.strip():
        context_text = ""
        text_input = f"Explain the content of this image."
    else:
        text_input = f"Explain the content of this image. The following context may help: {context_text}"

    print(f"Processing: {image_path} with context.")

    output = list(qwen_inference(media_input, text_input))  # Call inference function
    explanation = format_plain_text(output[-1])

    return explanation

In [ ]:
#split json file to max 50 nodes each file

def split_nodes(json_file):
    """Splits a JSON file into smaller JSON files, each containing a max of 50 nodes."""

    # Load the original JSON file
    with open(json_file, "r", encoding="utf-8") as file:
        data = json.load(file)

    if "nodes" not in data or not isinstance(data["nodes"], list):
        raise ValueError("Invalid JSON structure: 'nodes' list not found.")

    base_name = os.path.basename(json_file).replace(".json", "")
    json_folder = os.path.dirname(json_file)
    nodes = data["nodes"]
    output_files = []

    # Split nodes into chunks of 50
    for i in range(0, len(nodes), 50):
        chunk = nodes[i:i+50]

        # Re-index nodes (1 to 50, 51 to 100, etc.)
        for j, node in enumerate(chunk):
            node["index"] = i + j + 1  # Keeps original order but ensures sequential numbering per file

        # Create smaller JSON with updated metadata
        small_json = {
            "file_name": data["file_name"],
            "number_of_nodes": len(chunk),
            "nodes": chunk
        }

        small_json_file = os.path.join(json_folder, f"{base_name}_part_{(i//50) + 1}.json")

        # Save smaller JSON file
        with open(small_json_file, "w", encoding="utf-8") as file:
            json.dump(small_json, file, indent=4, ensure_ascii=False)

        output_files.append(small_json_file)
        print(f"Created: {small_json_file}")

    return output_files  # Return list of created JSON files


def get_context(image_path, nodes):
    """Finds the surrounding text (previous & next nodes) for an image node."""

    for i, node in enumerate(nodes):
        if "image_path" in node and node["image_path"] == image_path:  # Locate image node
            prev_text = nodes[i - 1]["text"] if i > 0 else ""  # Get previous text
            next_text = nodes[i + 1]["text"] if i < len(nodes) - 1 else ""  # Get next text
            return f"{prev_text} {next_text}".strip()  # Combine context

    return ""

In [ ]:
def process_split_files(split_files, json_folder):
    """Processes each split JSON file, finds images, and adds explanations."""

    processed_files = []

    for split_file in split_files:
        # Load the split JSON file
        with open(split_file, "r", encoding="utf-8") as file:
            data = json.load(file)

        updated = False  # Track if changes were made

        for node in data["nodes"]:
            if "image_path" in node and node["image_path"]:  # If node has an image
                image_path = os.path.join(json_folder, node["image_path"])

                if os.path.exists(image_path):  # Ensure image exists
                    context = get_context(image_path, data["nodes"])  # Get context

                    explanation = process_image(image_path, context, json_folder)
                    node["explanation"] = explanation  # Update node
                    updated = True  # Mark JSON as updated

        if updated:
            # Save the updated JSON with explanations
            processed_file = split_file.replace(".json", "_processed.json")
            with open(processed_file, "w", encoding="utf-8") as file:
                json.dump(data, file, indent=4, ensure_ascii=False)

            processed_files.append(processed_file)
            print(f"Processed and saved: {processed_file}")

    return processed_files

In [ ]:

def process_json_file(json_file):
    """Processes images from a JSON file and updates the JSON with explanations."""
    json_folder = os.path.dirname(json_file)

    # Load JSON file
    with open(json_file, "r", encoding="utf-8") as file:
        data = json.load(file)
    filename = os.path.basename(json_file).replace("_all_nodes_with_images.json", "")

    # image handling
    artifacts_folder = filename + "_artifacts"
    img_folder = os.path.join(json_folder, artifacts_folder)
    find_and_remove_duplicates(img_folder)
    remove_small_images(img_folder)

    # new json file with updated images & indexes
    updated_json = update_json(json_file, json_folder)

    torch.cuda.empty_cache()

    split_json = split_nodes(updated_json)
    processed_split = process_split_files(split_json, json_folder)

    merged_output_file = filename + "_qwen_processed.json"

    # Initialize the merged data structure
    merged_data = {"file_name": "merged_output", "number_of_nodes": 0, "nodes": []}

    # Iterate through each processed JSON file
    for processed_file in processed_split:
        with open(processed_file, "r", encoding="utf-8") as file:
            data = json.load(file)
            merged_data["nodes"].extend(data["nodes"])  # Append nodes

    # Update the total number of nodes
    merged_data["number_of_nodes"] = len(merged_data["nodes"])

    # Save the merged JSON file
    with open(merged_output_file, "w", encoding="utf-8") as file:
        json.dump(merged_data, file, indent=4, ensure_ascii=False)

    print(f"Merged JSON file saved as: {merged_output_file}")

    # Delete processed split files after merging
    for processed_file in processed_split:
        os.remove(processed_file)
        print(f"Deleted: {processed_file}")

    for processed_file in split_json:
        os.remove(processed_file)
        print(f"Deleted: {processed_file}")


In [ ]:
# process singular image
def process_single_image(image_path):
    """Generate explanation for an image using context from surrounding nodes."""
    media_input = image_path  # Construct full image path

    # Check if the image file exists
    if not os.path.exists(media_input):
      print(f"The file '{media_input}' does not exist.")
      return ""

    # Open image and check resolution
    with Image.open(media_input) as img:
        width, height = img.size
        if width < 50 and height < 50:  # Skip very small images
            print(f"The file '{media_input}' is too small to process.")
            return ""

    text_input = f"Explain the content of this image."
    print(f"Processing: {image_path}")

    output = list(qwen_inference(media_input, text_input))  # Call inference function
    explanation = format_plain_text(output[-1])

    return explanation

In [ ]:
compress_image("/content/output-docs/2014-monarch-plus-service-manual_artifacts/image_000000_86fd03e3bcbb1811667486cfaf784336b79a1cc05330327d18792d04aea05f44.png")
process_single_image("/content/output-docs/2014-monarch-plus-service-manual_artifacts/image_000000_86fd03e3bcbb1811667486cfaf784336b79a1cc05330327d18792d04aea05f44.png")

Compressed: /content/output-docs/2014-monarch-plus-service-manual_artifacts/image_000000_86fd03e3bcbb1811667486cfaf784336b79a1cc05330327d18792d04aea05f44.png to 47.29 KB
Processing: /content/output-docs/2014-monarch-plus-service-manual_artifacts/image_000000_86fd03e3bcbb1811667486cfaf784336b79a1cc05330327d18792d04aea05f44.png


'The image shows a RockShox fork, which is a type of suspension fork used in bicycles. The fork is designed to provide a smooth and controlled ride for the rider, especially on rough or uneven surfaces. The RockShox fork in the image is a 2017 model, as indicated by the year "2017" on the product label. The fork is black and appears to be made of metal, likely aluminum or steel, with a sleek and modern design. The fork has a dual shock system, which means it has two separate shock absorbers, one for each side of the fork. This design allows for better control and stability during high-speed riding. The fork also has a dropper post, which allows the rider to adjust the height of the fork\'s suspension, providing a more comfortable ride at different speeds and terrains. The brand "RockShox" is prominently displayed on the fork, indicating the manufacturer.'

In [ ]:
compress_image("/content/output-docs/SUPO-744_REV_A_artifacts/image_000015_bdae91db41c888e47741103302ea09139a75e6340655416cb6b1f3b9bfd3ec5c.png")
process_single_image("/content/output-docs/SUPO-744_REV_A_artifacts/image_000015_bdae91db41c888e47741103302ea09139a75e6340655416cb6b1f3b9bfd3ec5c.png")

Compressed: /content/output-docs/SUPO-744_REV_A_artifacts/image_000015_bdae91db41c888e47741103302ea09139a75e6340655416cb6b1f3b9bfd3ec5c.png to 44.45 KB
Processing: /content/output-docs/SUPO-744_REV_A_artifacts/image_000015_bdae91db41c888e47741103302ea09139a75e6340655416cb6b1f3b9bfd3ec5c.png


'The image provides a guide on how to read a battery display. Here is a detailed explanation of the content:\n\n1. **POWER LED**:\n   - **Green LED (ON)**: Indicates that the power is on.\n   - **Red LED (OFF)**: Indicates that the power is off.\n\n2. **BATTERY 1 STATUS**:\n   - **CHARGING**: The battery is charging.\n   - **READY**: The battery is fully charged.\n\n3. **POWER LED**:\n   - **Green LED (ON)**: Indicates that the power is on.\n   - **Red LED (OFF)**: Indicates that the power is off.\n\n4. **BATTERY 1 STATUS**:\n   - **CHARGING**: The battery is charging.\n   - **READY**: The battery is fully charged.\n\n5. **POWER LED**:\n   - **Green LED (ON)**: Indicates that the power is on.\n   - **Red LED (OFF)**: Indicates that the power is off.\n\n6. **BATTERY 1 STATUS**:\n   - **CHARGING**: The battery is charging.\n   - **READY**: The battery is fully charged.\n\nThe image is divided into two sections, each representing a different battery. The left section shows the power LED a

In [ ]:
process_json_file("/content/output-docs/2014-monarch-plus-service-manual_all_nodes_with_images.json")

Total duplicates removed: 0
Total small images removed: 0
Updated JSON saved as: /content/output-docs/2014-monarch-plus-service-manual_all_nodes_with_images_updated.json
Created: /content/output-docs/2014-monarch-plus-service-manual_all_nodes_with_images_updated_part_1.json
Created: /content/output-docs/2014-monarch-plus-service-manual_all_nodes_with_images_updated_part_2.json
Created: /content/output-docs/2014-monarch-plus-service-manual_all_nodes_with_images_updated_part_3.json
Created: /content/output-docs/2014-monarch-plus-service-manual_all_nodes_with_images_updated_part_4.json
Created: /content/output-docs/2014-monarch-plus-service-manual_all_nodes_with_images_updated_part_5.json
Created: /content/output-docs/2014-monarch-plus-service-manual_all_nodes_with_images_updated_part_6.json
Compressed: /content/output-docs/2014-monarch-plus-service-manual_artifacts/image_000000_86fd03e3bcbb1811667486cfaf784336b79a1cc05330327d18792d04aea05f44.png to 47.29 KB
Processing: /content/output-do

In [ ]:
process_image()